BiLSTM序列标注---盲汉翻译

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random # for dataset shuffling
import openpyxl # for recording experimental value
import copy # for data shuffling

https://blog.csdn.net/vivian_ll/article/details/93894151

In [3]:
# GPU training
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
    print("number of device",torch.cuda.device_count())
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [4]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # LSTM以word_embeddings作为输入, 输出维度为 hidden_dim 的隐藏状态值
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # 线性层将隐藏状态空间映射到标注空间
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # 各个维度的含义是 (num_layers*num_directions, batch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [5]:
#Making training data
original_data = []
t = open('./预测结果.txt',"r", encoding='UTF8')
f = open("./输入_无标调拼音.txt", "r", encoding="UTF8")
#计数 数一共有多少行
lines = f.readlines()
tags = t.readlines()

# Training data: 6692(total number of sentences)*2(line&tag)
for index, line in enumerate(lines):
    line = lines[index].rstrip().split()
    tag = list(tags[index])[:-1] # To delete the last /n
    original_data.append((line, tag))

In [6]:
word_to_ix = {}
for sent, tags in original_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

np.save('word_to_ix.npy',word_to_ix)
word_to_ix

{'ai': 0,
 'ya': 1,
 ',': 2,
 'dui': 3,
 'bu': 4,
 'qi': 5,
 'wo': 6,
 'ba': 7,
 'zhe': 8,
 'xie': 9,
 'shi': 10,
 'qing': 11,
 'tong': 12,
 'wang': 13,
 'le': 14,
 '.': 15,
 'shou': 16,
 'biao': 17,
 'zai': 18,
 'jiao': 19,
 'ke': 20,
 'xue': 21,
 'jia': 22,
 'men': 23,
 'zheng': 24,
 'yan': 25,
 'jiu': 26,
 'zhi': 27,
 'liao': 28,
 'zi': 29,
 'bing': 30,
 'de': 31,
 'yao': 32,
 'wu': 33,
 'shen': 34,
 'ye': 35,
 'er': 36,
 'gong': 37,
 'zuo': 38,
 'si': 39,
 'duo': 40,
 'nian': 41,
 'ren': 42,
 'dou': 43,
 'hen': 44,
 'dai': 45,
 'ta': 46,
 'yi': 47,
 'ge': 48,
 'hao': 49,
 'da': 50,
 'guo': 51,
 'zhu': 52,
 'jing': 53,
 'ying': 54,
 'gai': 55,
 'fa': 56,
 'yang': 57,
 'mei': 58,
 'quan': 59,
 'jie': 60,
 'he': 61,
 'ping': 62,
 'xi': 63,
 'sheng': 64,
 'huo': 65,
 'an': 66,
 'ding': 67,
 'ti': 68,
 'yu': 69,
 'yun': 70,
 'dong': 71,
 'SOti': 72,
 'hui': 73,
 'zhun': 74,
 'bei': 75,
 'xiao': 76,
 'jun': 77,
 'wei': 78,
 'yuan': 79,
 'guan': 80,
 'xin': 81,
 'hu': 82,
 'wen': 83,
 'nu

In [7]:
tag_to_ix = {}
for sent, tags in original_data:
    for word in tags:
        if word not in tag_to_ix:
            tag_to_ix[word] = len(tag_to_ix)
np.save('tag_to_ix.npy',tag_to_ix)
tag_to_ix

{'哎': 0,
 '呀': 1,
 ',': 2,
 '对': 3,
 '不': 4,
 '起': 5,
 '我': 6,
 '把': 7,
 '这': 8,
 '些': 9,
 '事': 10,
 '情': 11,
 '统': 12,
 '忘': 13,
 '了': 14,
 '.': 15,
 '手': 16,
 '表': 17,
 '在': 18,
 '教': 19,
 '室': 20,
 '科': 21,
 '学': 22,
 '家': 23,
 '们': 24,
 '正': 25,
 '研': 26,
 '究': 27,
 '治': 28,
 '疗': 29,
 '艾': 30,
 '滋': 31,
 '病': 32,
 '的': 33,
 '药': 34,
 '物': 35,
 '沈': 36,
 '爷': 37,
 '儿': 38,
 '工': 39,
 '作': 40,
 '四': 41,
 '十': 42,
 '多': 43,
 '年': 44,
 '人': 45,
 '都': 46,
 '很': 47,
 '爱': 48,
 '戴': 49,
 '他': 50,
 '是': 51,
 '一': 52,
 '个': 53,
 '好': 54,
 '大': 55,
 '国': 56,
 '主': 57,
 '义': 58,
 '精': 59,
 '神': 60,
 '应': 61,
 '该': 62,
 '发': 63,
 '扬': 64,
 '每': 65,
 '样': 66,
 '全': 67,
 '世': 68,
 '界': 69,
 '和': 70,
 '平': 71,
 '希': 72,
 '望': 73,
 '生': 74,
 '活': 75,
 '安': 76,
 '定': 77,
 '体': 78,
 '育': 79,
 '运': 80,
 '动': 81,
 '者': 82,
 '协': 83,
 '会': 84,
 '准': 85,
 '备': 86,
 '吸': 87,
 '收': 88,
 '杨': 89,
 '小': 90,
 '君': 91,
 '为': 92,
 '员': 93,
 '关': 94,
 '心': 95,
 '护': 96,
 '温': 97,
 '暖': 98,
 '老': 99,
 '特': 100,

In [8]:
original_data[1][0]

['ai',
 'ya',
 ',',
 'wo',
 'ba',
 'shou',
 'biao',
 'wang',
 'zai',
 'jiao',
 'shi',
 'le',
 '.']

In [9]:
# Training the LSTM

#Monitor trianing results
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

#Helper function to convert data into tensor
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

# Accuracy recording in excel
workbook = openpyxl.Workbook()
worksheet = workbook.active



#Change parameters to reach better performances
for i in range(10):
    # record results.
    acc_repeat_train = ['acc_train']
    acc_repeat_val = ["acc_val"]
    loss_repeat_train = ["loss_repeat_train"]
    loss_repeat_val = ["loss_repeat_val"]

    # Repeat for three times in new model with different data. 
    for repeat in range(3):
        # Initialize the training model
        EMBEDDING_DIM = 60 
        HIDDEN_DIM = 10 + 20*i
        EPOCH=30
        model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
        model = model.to(device) # For GPU calculation
        loss_function = nn.NLLLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

        #Set data writer
        g = "./log/"+"Hidden_dim_"+str(EMBEDDING_DIM)+"_Embedding_dim_"+str(HIDDEN_DIM)+"repeat_"+str(repeat) 
        writer = SummaryWriter(g)

        #Shuffle data
        random.seed(repeat)
        training_data = copy.deepcopy(original_data) #Deep copy to make sure the original data is not influenced.
        random.shuffle(training_data)

        #Record accuracy


        for epoch in range(EPOCH):  
            print(epoch)
            training_acuracy = []
            validation_acuracy = []
            loss_list_train = []
            loss_list_validation = []
            for sentence, tags in tqdm(training_data[:5000]):
                # Clear Gradient because PyTorch accumulate it
                model.zero_grad()
                # Clear hidden state for LSTM, seperate them from the last instance.
                model.hidden = model.init_hidden()

                # 准备网络输入, 将其变为词索引的 Tensor 类型数据            
                sentence_in = prepare_sequence(sentence, word_to_ix)
                targets = prepare_sequence(tags, tag_to_ix)
    
                # Forward propagation
                tag_scores = model(sentence_in)

                # 第四步: 计算损失和梯度值, 通过调用 optimizer.step() 来更新梯度
                loss = loss_function(tag_scores, targets)
                a = (torch.argmax(tag_scores, dim=-1) == targets).sum().item() / len(targets)
                training_acuracy.append(a)
                loss.backward()
                optimizer.step()
                loss_list_train.append(loss.item())
            
            #Validation
            for sentence, tags in tqdm(training_data[5000:6000]):
                model.zero_grad()
                model.hidden = model.init_hidden()
                # 准备网络输入, 将其变为词索引的 Tensor 类型数据
                sentence_in = prepare_sequence(sentence, word_to_ix)
                targets = prepare_sequence(tags, tag_to_ix)
                # 第三步: 前向传播.
                tag_scores = model(sentence_in)
                #计算损失
                a = (torch.argmax(tag_scores, dim=-1) == targets).sum().item() / len(targets)
                validation_acuracy.append(a)
                loss = loss_function(tag_scores, targets)
                loss_list_validation.append(loss.item())
            
                
            #LSTM 可以做padding
            
            # Calculate error and accuracy
            # print(loss)
            loss_train= sum(loss_list_train)
            writer.add_scalar('Loss/training', loss_train, epoch)
            loss_val = sum(loss_list_validation)
            writer.add_scalar('Loss/validation', loss_val, epoch)
            #print(torch.argmax(tag_scores, dim=-1))
            # print("training_acuracy",sum(training_acuracy)/len(training_acuracy))
            acc_train = sum(training_acuracy)/len(training_acuracy)
            writer.add_scalar('Accuracy/training', acc_train, epoch)
            # print("validation_acuracy:",sum(validation_acuracy)/len(validation_acuracy))
            acc_val =sum(validation_acuracy)/len(validation_acuracy) 
            writer.add_scalar('Accuracy/validation', acc_val, epoch)

        acc_repeat_train.append(acc_train)
        acc_repeat_val.append(acc_val)
        loss_repeat_train.append(loss_train)
        loss_repeat_val.append(loss_val)         
            
        # 查看训练后的得分
        # with torch.no_grad():
        #     inputs = prepare_sequence(training_data[0][0], word_to_ix)
        #     tag_scores = model(inputs)

    worksheet.append(acc_repeat_train+acc_repeat_val+loss_repeat_train+loss_repeat_val)
writer.close()
workbook.save("Result.xlsx")

0


  0%|          | 14/5000 [00:00<04:12, 19.73it/s]


ValueError: Expected input batch_size (10) to match target batch_size (13).

In [12]:
for i in original_data:
    if len(i[0])!=len(i[1]):
        print(i[1])
        print(i[0])

['北', '方', '人', '喜', '欢', '吃', '面', '食', ',', '比', '如', '饺', '子', '包', '子', '面', '条', '馒', '头', ',', '南', '方', '人', '喜', '欢', '吃', '米', '饭', '.']
['beifangren', 'xihuan', 'chi', 'mianshi', ',', 'biru', 'jiaozi', 'baozi', 'miantiao', 'mantou', ',', 'nanfangren', 'xihuan', 'chi', 'mifan', '.']
['他', '们', '的', '保', '密', '工', '作', '做', '得', '很', '好', ',', '到', '现', '在', '还', '没', '有', '人', '知', '道', '明', '天', '他', '们', '结', '婚', '的', '事', '情', '.']
['tamen', 'de', 'baomi', 'gongzuo', 'zuo', 'de', 'henhao', ',', 'dao', 'xianzai', 'hai', 'meiyou', 'ren', 'zhidao', 'mingtian', 'tamen', 'jiehun', 'de', 'shiqing', '.']
['"', '西', '安', '事', '变', '"', '以', '后', ',', '蒋', '介', '石', '被', '迫', '发', '表', '宣', '言', ',', '联', '合', '抗', '日', '.']
['"xi', 'an', 'shi', 'bian', '"yi', 'hou', ',', 'jiang', 'jie', 'shi', 'bei', 'po', 'fa', 'biao', 'xuan', 'yan', ',', 'lian', 'he', 'kang', 'ri', '.']
['爸', '爸', '本', '来', '说', '带', '我', '去', '旅', '行', ',', '可', '是', '今', '天', '又', '说', '不', '带', '我', '去', '了',

In [ ]:
original_data[400]

(['2',
  '4',
  '3',
  '1',
  '1',
  '2',
  '3',
  '4',
  '2',
  '4',
  ',',
  '4',
  '3',
  '2',
  '4',
  '0',
  '1',
  '2',
  '.'],
 ['疼',
  '痛',
  '使',
  '他',
  '终',
  '于',
  '忍',
  '耐',
  '不',
  '住',
  ',',
  '跳',
  '起',
  '来',
  '叫',
  '了',
  '出',
  '来',
  '.'])

In [10]:
print(tags)
print(sentence)
f.close()
t.close()

['阿', '正', '传', '中', '只', '有', '一', '个', '主', '要', '人', '物', '.']
['Qzhong', 'zhi', 'you', 'yi', 'ge', 'zhu', 'yao', 'ren', 'wu', '.']


['小', '李', '在', '陌', '生', '人', '面', '前', '显', '得', '很', '拘', '束', '.']